<a href="https://colab.research.google.com/github/Chirag314/Tensorflow-practice/blob/main/Tensorboard_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Taking TensorFlow to Production

In [1]:
# Import Python packages
import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from xgboost import XGBRegressor
from xgboost import plot_tree
from xgboost import plot_importance
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, KFold, cross_val_score,TimeSeriesSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import itertools
import tensorflow as tf
import numpy as np
import datetime
from sklearn import metrics

Step 3: Load the data

In [6]:
from matplotlib import image
# We will now load MNIST data from keras
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data()
x_train=x_train.reshape(-1,28,28,1)
x_test=x_test.reshape(-1,28,28,1)
# Padd images by 2 pixels
x_train=np.pad(x_train,((0,0),(2,2),(2,2),(0,0)),'constant')
x_test=np.pad(x_test,((0,0),(2,2),(2,2),(0,0)),'constant')

# Normalize 
x_train=x_train/255
y_test=y_test/255

# Set model parameters
image_width=x_train[0].shape[0]
image_height=x_train[0].shape[1]
num_channels=1

batch_size=100
evaluation_size=500
generations=300
eval_every=5

seed=98
np.random.seed(seed)
tf.random.set_seed(seed)

#Declare the model
input_data=tf.keras.Input(dtype=tf.float32,shape=(image_width,image_height,num_channels),name='INPUT')

conv1=tf.keras.layers.Conv2D(filters=6,
                             kernel_size=5,
                             padding='VALID',
                             activation='relu',
                             name="C1")(input_data)
max_pool1=tf.keras.layers.MaxPool2D(pool_size=2,
                                   strides=2,
                                   padding='SAME',
                                   name='S1')(conv1)
conv2=tf.keras.layers.Conv2D(filters=16,
                             kernel_size=5,
                             strides=1,
                             padding='VALID',
                             activation='relu',
                             name="C3")(max_pool1)
max_pool2=tf.keras.layers.MaxPool2D(pool_size=2,
                                   strides=2,
                                   padding='SAME',
                                   name='S4')(conv2)
flatten=tf.keras.layers.Flatten(name='FLATTEN')(max_pool2)
fully_connected1=tf.keras.layers.Dense(units=120,
                                       activation='relu',
                                       name='F5')(flatten)

fully_connected2=tf.keras.layers.Dense(units=84,
                                       activation='relu',
                                       name='F6')(fully_connected1)
final_model_output=tf.keras.layers.Dense(units=10,
                                       activation='relu',
                                       name='OUTPUT')(fully_connected2)

model=tf.keras.Model(inputs=input_data,outputs=final_model_output)


In [7]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [8]:
log_dir="logs/experiment-" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 

In [9]:
#Next, we will instantiate a TensorBoard callback and pass it to the fit method. All logs during the training phase will be stored in this directory and can be viewed instantly in TensorBoard:
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=log_dir,
                                                    write_images=True,
                                                    histogram_freq=1)

model.fit(x_train,
          y_train,
          epochs=5,
          validation_data=(x_test,y_test),
          callbacks=[tensorboard_callback])

Epoch 1/5
1875/1875 [==============================] - 48s 25ms/step - loss: 2.3985 - accuracy: 0.1086 - val_loss: 2.3190 - val_accuracy: 0.0980
Epoch 2/5
1875/1875 [==============================] - 49s 26ms/step - loss: 2.3026 - accuracy: 0.0987 - val_loss: 2.3190 - val_accuracy: 0.0980
Epoch 3/5
1875/1875 [==============================] - 46s 24ms/step - loss: 2.3026 - accuracy: 0.0987 - val_loss: 2.3190 - val_accuracy: 0.0980
Epoch 4/5
1875/1875 [==============================] - 47s 25ms/step - loss: 2.3026 - accuracy: 0.0987 - val_loss: 2.3190 - val_accuracy: 0.0980
Epoch 5/5
1875/1875 [==============================] - 48s 26ms/step - loss: 2.3026 - accuracy: 0.0987 - val_loss: 2.3190 - val_accuracy: 0.0980


In [ ]:
#start the TensorBoard application by running the following command